In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
os.mkdir('/kaggle/working/artifacts')

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 

# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
books = pd.read_csv('/kaggle/input/book-recommendation-dataset/Books.csv')

In [ ]:
books.head()

In [ ]:
books.shape

In [ ]:
books.columns

In [ ]:
books = books[['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-L']]

In [ ]:
books.head(2)

In [ ]:
books.rename(columns={
    "Book-Title": "title",
    "Book-Author": "author",
    "Year-Of-Publication": "year",
    "Publisher": "publisher",
    "Image-URL-L": "img_url"}, inplace = True)

In [ ]:
books.head(3)

In [ ]:
users = pd.read_csv('/kaggle/input/book-recommendation-dataset/Users.csv')

In [ ]:
users.head()

In [ ]:
users.shape

In [ ]:
ratings = pd.read_csv('/kaggle/input/book-recommendation-dataset/Ratings.csv')

In [ ]:
ratings.head()

In [ ]:
ratings.rename(columns={
    "User-ID": "user_id",
    "Book-Rating": "rating"
}, inplace = True)

In [ ]:
ratings.head()

In [ ]:
ratings['user_id'].value_counts()

In [ ]:
ratings['user_id'].unique().shape

In [ ]:
x = ratings['user_id'].value_counts() > 200

In [ ]:
x[x].shape

In [ ]:
y = x[x].index

In [ ]:
y

In [ ]:
ratings =ratings[ratings['user_id'].isin(y)]

In [ ]:
ratings.head()

In [ ]:
ratings.shape

In [ ]:
ratings_with_books = ratings.merge(books, on = "ISBN")

In [ ]:
ratings_with_books.head()

In [ ]:
ratings_with_books.shape

In [ ]:
num_rating = ratings_with_books.groupby('title')['rating'].count().reset_index()

In [ ]:
num_rating.head()

In [ ]:
num_rating.rename(columns = {"rating": "num_of_rating"}, inplace=True)

In [ ]:
num_rating.head()

In [ ]:
final_rating = ratings_with_books.merge(num_rating, on= 'title')

In [ ]:
final_rating.head(2)

In [ ]:
final_rating.shape

In [ ]:
final_rating = final_rating[final_rating['num_of_rating']>=50]

In [ ]:
final_rating.sample(10)

In [ ]:
final_rating.shape

In [ ]:
final_rating.drop_duplicates(['user_id','title'], inplace = True)

In [ ]:
final_rating.shape

In [ ]:
book_pivot = final_rating.pivot_table(columns='user_id', index='title', values= 'rating')

In [ ]:
book_pivot

In [ ]:
book_pivot.shape

In [ ]:
book_pivot.fillna(0, inplace=True)

In [ ]:
book_pivot

In [ ]:
from scipy.sparse import csr_matrix

In [ ]:
book_sparse = csr_matrix(book_pivot)

In [ ]:
book_sparse

In [ ]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(algorithm= 'brute')

In [ ]:
model.fit(book_sparse)

In [ ]:
distance, suggestion = model.kneighbors(book_pivot.iloc[237,:].values.reshape(1,-1), n_neighbors=6)

In [ ]:
distance

In [ ]:
suggestion

In [ ]:
for i in range(len(suggestion)):
    print(book_pivot.index[suggestion[i]])

In [ ]:
book_pivot.index[3]

In [ ]:
book_pivot.index

In [ ]:
books_name = book_pivot.index

In [ ]:
import pickle
pickle.dump(model, open('/kaggle/working/artifacts/model.pkl', 'wb'))
pickle.dump(books_name, open('/kaggle/working/artifacts/books_name.pkl', 'wb'))
pickle.dump(final_rating, open('/kaggle/working/artifacts/final_rating.pkl', 'wb'))
pickle.dump(book_pivot, open('/kaggle/working/artifacts/book_pivot.pkl', 'wb'))

In [ ]:
def recommend_book(book_name) :
    book_id = np.where(book_pivot.index == book_name)[0][0]
    distance, suggestion = model.kneighbors(book_pivot.iloc[book_id,:].values.reshape(1,-1), n_neighbors=6)
    
    for i in range(len(suggestion)):
        books = book_pivot.index[suggestion[i]]
        for j in books:
            print(j)

In [ ]:
book_name = 'A Bend in the Road'
recommend_book(book_name)